In [ ]:
import pandas as pd
import io
import csv
import docx
import datetime as datetime
import numpy as np

def read_docx_tables(doc, tab_id=None, **kwargs):

    def read_docx_tab(tab, **kwargs):
        vf = io.StringIO()
        writer = csv.writer(vf)
        for row in tab.rows:
            writer.writerow(cell.text for cell in row.cells)
        vf.seek(0)
        if len(tab.rows)>0:
            return pd.read_csv(vf, **kwargs)
        else:
            return pd.DataFrame()

    if tab_id is None:
        return [read_docx_tab(tab, **kwargs) for tab in doc.tables]
    else:
        return read_docx_tab(doc.tables[tab_id], **kwargs)
    
def remove_row(table, row):
    tbl = table._tbl
    tr = row._tr
    tbl.remove(tr)

def log_table(doc, table_name, comments=[]):
    
    if len(comments)==0:
        table=doc.add_table(0,0)
    
    else:
    
        table=doc.add_table(1,2)
        table.style = 'ColorfulList-Accent2'

        if table_name == "goals":
            heading_cells=table.rows[0].cells
            heading_cells[0].text= "Goals"
            heading_cells[1].text= "Summary"

        if table_name == "accomplished":
            heading_cells=table.rows[0].cells
            heading_cells[0].text= "Accomplished"
            heading_cells[1].text= "Notes"
            
        if table_name == "pushes":
            heading_cells=table.rows[0].cells
            heading_cells[0].text= "Commit"
            heading_cells[1].text= "Comment"
            
        if table_name == "to_do":
            heading_cells=table.rows[0].cells
            heading_cells[0].text= "To Do"
            heading_cells[1].text= "Notes"
            

        for row in comments:
            cells=table.add_row().cells
            cells[0].text = row[0]
            cells[1].text = row[1]
            
        doc.add_paragraph()
        
def new_entry(doc, today, goals, accomplished, pushes, to_do):
    heading=doc.add_paragraph(date)

    log_table(doc,"goals",goals)

    log_table(doc,"accomplished",accomplished)

    log_table(doc,"pushes",pushes)

    log_table(doc,"to_do",to_do)

def table_merge(doc, table, comments, name,i):
    if len(table) == 0 and len(comments) > 0:
        log_table(doc, name, comments)
    
    elif len(table) == 0 and len(comments) == 0:
        return
    
    else:
        if name == 'goals':
            columns=["Goals","Summary"]
        elif name == 'accomplished':
            columns=["Accomplished","Notes"]
        elif name == 'pushes':
            columns=["Commit","Comment"]    
        elif name == 'to_do':
            columns=["To Do","Notes"]    
        
        update=pd.DataFrame(comments,columns=columns)

        pd.merge(table.replace(np.NaN,' '),update,'right')
                
        for ind, row in enumerate(doc.tables[i].rows):
            if ind>0:
                doc.tables[i].row_cells(ind)[0].text = comments[ind-1][0]
                doc.tables[i].row_cells(ind)[1].text = comments[ind-1][1]
                
    
def update_entry(doc, today, goals, accomplished, pushes, to_do):
    for i, tab in enumerate(doc.tables):
        if i % 4 == 0:
            goal_ind = i
        elif i % 4 == 1:
            acc_ind = i
        elif i % 4 == 2:
            push_ind = i
        elif i % 4 == 3:
            td_ind = i
        
    goal_tab = read_docx_tables(doc,goal_ind)
    acc_tab = read_docx_tables(doc,acc_ind)
    push_tab = read_docx_tables(doc,push_ind)
    td_tab = read_docx_tables(doc,td_ind)      
    
    table_merge(doc, goal_tab, goals, 'goals', goal_ind)
    table_merge(doc, acc_tab, accomplished, 'accomplished', acc_ind)
    table_merge(doc, push_tab, pushes, 'pushes',push_ind)
    table_merge(doc, td_tab, to_do, 'to_do',td_ind)
    
    

In [ ]:
date=datetime.datetime.now().strftime('%Y-%m-%d')

goals=[
        ["Update Logging Script with Git and Code Output",""],
        ["Include timestamp and project title boxes in logging",""],
       ["Read Papers",""]
]
accomplished=[
]
to_do=[
]
pushes=[]

In [ ]:
test=docx.Document("./log_test.docx")
for i, par in enumerate(test.paragraphs):
    if i % 4 == 1:
        date_par = test.paragraphs[i].text
if date_par != date:
    new_entry(test, date, goals, accomplished, pushes, to_do)
    test.save('log_test.docx')
    print("New Entry Added")
if date_par == date:
    update_entry(test, date, goals, accomplished, pushes, to_do)
    test.save('log_test.docx')
    print("Updated Entry")

In [1]:
from git import Repo
repo_dir = 